**Consult the README.md for installation instructions.**

In [1]:
from flagcomplexSvg import FlagComplexSvg
from flagcomplex.ProjGeometryUtility import transform_four_points
import numpy as np
import copy

# Eruption flow on a four-tuple of flags
First, we will define a flag complex with four flags.

In [2]:
fcomplex = FlagComplexSvg()

other_points = [np.array([1, 1, 1]), np.array([1, -1, 1]), np.array([-1, -1, 1]), np.array([-1, 1, 1])]
p0 = np.array([0, 1, 1])
q0 = other_points[0]

p1 = np.array([1, 0, 1])
q1 = other_points[1]

p2 = np.array([0, -1, 1])
q2 = other_points[2]

p3 = np.array([-1, 0, 1])
q3 = other_points[3]


fcomplex.add_flag(p0,q0)
fcomplex.add_flag(p1,q1)
fcomplex.add_flag(p2,q2)
fcomplex.add_flag(p3,q3)

fcomplex.set_projection_plane(np.array([0,0,1]))

In [3]:
d = fcomplex.initialize_canvas()

fcomplex.create_triangulation()
fcomplex.draw_complex()

#d1 = copy.deepcopy(d)
#fcomplex.visualize(d1,with_helper_lines=True, with_inner_triangles = False, with_label= True)
#d1.savePng("ef_4.png")

It is important to note that applying the eruption flow to one triangle also transforms the points of the other triangle: For defining the eruption flow, we divide the triangle into three quadrilaterals $□(q_i, p_{i-1}, p_{i+1}, u_i)$  and one smaller triangle $\Delta(u_0, u_1, u_2)$. If a point of another flag lies in one of the quadrilaterals, it must be transformed accordingly.

Therefore, we have to define `fcomplex.set_subdivision` in order to specify in which quadrilateral a point lies in.

In order to obtain a nicer visualization, we can retransform the whole complex with a projective transformation. In our case, we bring the `fcomplex.qs` back to where we started.

In [4]:
steps=100
step_width = 0.1

triangle0 = [1, 2, 0]
triangle1 = [2, 3, 0]

fcomplex.set_subdivision([2, 3, 0], {2: [0], 3: [2, 1], 0: [3]})
fcomplex.set_subdivision([1, 2, 0], {1: [0, 3], 2: [1], 0: [2]})


style = "Q"

fcomplex.erupt_complex_along_triangle(t= -10, triangle=triangle1, transformation_style=style)
fcomplex.erupt_complex_along_triangle(t= +10, triangle=triangle0, transformation_style=style)

for r in range(1, steps):
    fcomplex.erupt_complex_along_triangle(t=  step_width, triangle=triangle1, transformation_style=style)
    fcomplex.erupt_complex_along_triangle(t= -step_width, triangle=triangle0, transformation_style=style)
    fcomplex.projective_transformation = transform_four_points(fcomplex.qs, [other_points[i-1] for i in range(4)])
  
    d1 = copy.deepcopy(d)

    fcomplex.visualize(d1,with_helper_lines=False, with_inner_triangles = False, with_label=True)

    d1.savePng("./temp/ef4_" + str(r) + ".png")

In [5]:
from moviepy.editor import ImageSequenceClip

images_list = ["temp/ef4_"+str(r)+".png" for r in range(1,steps)]
animation = ImageSequenceClip(images_list, fps=25)
#animation.write_videofile("eruption_flow_movie_without_tesselation.mp4")
#animation.write_gif("eruption_flow_style_p.gif")
animation.ipython_display(fps=25, loop=True, autoplay=True)

t:   4%|▍         | 4/100 [00:00<00:03, 28.66it/s, now=None]

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


In [6]:
fcomplex.triangle_subdivision

{frozenset({0, 2, 3}): {2: [0], 3: [2, 1], 0: [3]},
 frozenset({0, 1, 2}): {1: [0, 3], 2: [1], 0: [2]}}